# Getting Recommendations

Minim can help you discover new artists and music by leveraging the Spotify and TIDAL recommender systems and suggesting tracks based on your music libraries.

In [ ]:
from base64 import b64encode
import random

from IPython.display import HTML, IFrame, display
from ipywidgets import Output, GridspecLayout

from minim import spotify, tidal

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

## Spotify

In the Spotify Web API, you can use `minim.spotify.WebAPI.get_recommendations()` to generate track recommendations based on your favorite artists, genres, and/or tracks, and a number of tunable track attributes.

In the following example, we will generate recommendations using only seed tracks. 

First, we create a Spotify Web API client by instantiating a `minim.spotify.WebAPI` object:

In [ ]:
client_spotify = spotify.WebAPI()

If you want to access your Spotify library for seed artists and/or tracks, make sure that the appropriate credentials and scopes are passed to the constructor above, stored in environment variables, or available in the Minim configuration file.

:::{seealso}
See [Getting Started](../getting_started.ipynb) for more information about setting up clients with user authentication.
:::

The seed tracks can either come from your favorite Spotify tracks:

```python
seed_tracks = [track["track"]["id"] for track in client_spotify.get_saved_tracks(limit=50)["items"]]
```

or be specified manually:

In [ ]:
seed_tracks = [
    "0JZ9TvOLtZJaGqIyC4hYZX",   # Avicii - Trouble
    "0bmB3nzQuHBfI6nM4SETVu",   # Cash Cash - Surrender
    "1PQ8ywTy9V2iVZWJ7Gyxxb",   # Mako - Our Story
    "70IFLb5egLA8WUFWgxBoRz",   # Mike Williams - Fallin' In
    "6jSPbxZLd2yemJTjz2gqOT",   # Passion Pit & Galantis - I Found U
    "76B6LjxTolaSGXLANjNndR",   # Sick Individuals - Made for This
    "2V65y3PX4DkRhy1djlxd9p",   # Swedish House Mafia - Don't You Worry Child (feat. John Martin)
    "1gpF8IwQQj8qOeVjHfIIDU"    # Zedd - Find You (feat. Matthew Koma & Miriam Bryant)
]

Since we are limited to 5 seed tracks, we will randomly select 5 tracks from our `list` of seed tracks and pass them as a keyword argument to `minim.spotify.WebAPI.get_recommendations()`:

In [ ]:
recommended_tracks = client_spotify.get_recommendations(
    seed_tracks=random.choices(seed_tracks, k=5)
)["tracks"]

Finally, we can add the recommended tracks to a new private Spotify playlist so that they can be accessed from another device:

```python
spotify_playlist = client_spotify.create_playlist("Minim Mix", public=False)
client_spotify.add_playlist_items(
    spotify_playlist["id"], 
    [f"spotify:track:{track['id']}" for track in recommended_tracks]
)
with open(globals()["_dh"][0].parents[3] / "assets/minim_mix_small.jpg", "rb") as f:
    client_spotify.add_playlist_cover_image(spotify_playlist["id"], b64encode(f.read()))
```

The last two lines above add a nifty custom cover art for mixes created with Minim:

![Minim Mix cover art](https://raw.githubusercontent.com/bbye98/minim/main/assets/minim_mix_small.jpg)

If you are building an interactive or web application, you can instead visualize the recommended tracks using embeds:

In [ ]:
grid = GridspecLayout(len(recommended_tracks), 1)
for i, track in enumerate(recommended_tracks):
    out = Output()
    with out:
        display(IFrame(f"https://open.spotify.com/embed/track/{track['id']}", 
                       frameBorder=0, loading="lazy", height=152, width=510))
    grid[*divmod(i, 1)] = out
grid

## TIDAL

In the TIDAL API, you can use `minim.tidal.API.get_similar_albums()`, `minim.tidal.API.get_similar_artists()`, and `minim.tidal.API.get_similar_tracks()` to generate recommendations based on your favorite albums, artists, and tracks, respectively.

In the following example, we will discover tracks similar to our favorite tracks only since the procedure for generating album and artist recommendations is similar.

First, we create a TIDAL API client by instantiating a `minim.tidal.API` object:

In [ ]:
client_tidal = tidal.API()

and specify the tracks for which to find similar tracks:

In [ ]:
favorite_tracks = [
    51073951,   # Avicii - Trouble
    62082351,   # Cash Cash - Surrender
    32553484,   # Mako - Our Story
    147258423,  # Mike Williams - Fallin' In
    109273852,  # Passion Pit & Galantis - I Found U
    237059212,  # Sick Individuals - Made for This
    17271290,   # Swedish House Mafia - Don't You Worry Child (feat. John Martin)
    27171015    # Zedd - Find You (feat. Matthew Koma & Miriam Bryant)
]

Then, we randomly select a track from our `list` of favorite tracks and pass it to `minim.tidal.API.get_similar_tracks()`:

In [ ]:
similar_tracks = client_tidal.get_similar_tracks(random.choice(favorite_tracks), 
                                                 "US")["data"]

Finally, we can display the similar tracks interactively using embeds:

In [ ]:
grid = GridspecLayout(len(similar_tracks) // 2, 2)
for i, track in enumerate(similar_tracks):
    out = Output()
    with out:
        display(
            HTML('<div class="tidal-embed" '
                 'style="position:relative;padding-bottom:100%;'
                 'height:0;overflow:hidden;max-width:100%">'
                 '<iframe src="https://embed.tidal.com/tracks/'
                 f'{track["resource"]["id"]}?layout=gridify" '
                 'allowfullscreen="allowfullscreen" frameborder="0" '
                 'style="position:absolute;top:0;left:0;width:100%;'
                 'height:1px;min-height:100%;margin:0 auto">'
                 '</iframe></div>')
        )
    grid[*divmod(i, 2)] = out
grid